INF8770 Technologies multimédias

Polytechnique Montréal

Importation des librairies

In [ ]:
from PIL import Image
import os
from skimage.feature.texture import local_binary_pattern
import glob
import cv2
import numpy

 Matrices de Co-occurrence (MC)

In [ ]:
def desc_MC(imagePath, dx, dy, color1, color2):
  image = Image.open(imagePath)
  width = len(image)
  height = len(image[0])
  result = []

  for column in range(0, width):
    if column + dx >= width:
      break
    for row in range(0, height):
      if row + dy >= height: 
        break
      areCorrespondingColors = True
      for canal in range(0, 3):
        if image[column][row][canal] != color1[canal] or image[column + dx][row + dy][canal] != color2[canal]:
           areCorrespondingColors = False
           break
      result += 1 if areCorrespondingColors else 0

  return result

Modèles Binaires Locaux (LBP)

In [ ]:
longueurCodes = 8
rayon = 1

def desc_LBP(path):
  image = cv2.imread(path,cv2.IMREAD_GRAYSCALE)
  desc =  local_binary_pattern(image, longueurCodes, rayon)
  return numpy.asarray(desc, numpy.float32)

Importation des images

In [ ]:
query = glob.glob(os.path.join("./data", "*.jpg"))
database = glob.glob(os.path.join("./data/database", "*.jpg"))

Méthode de comparaison

In [ ]:
def intersection(vector_i, vector_j):
    return numpy.sum(numpy.min(vector_i, vector_j))

def norms(vector_i, vector_j):
    return numpy.sum(numpy.abs(numpy.diff(vector_i, vector_j)))

def bhattacharyya(vector_i, vector_j):
    return -numpy.log((numpy.sum(numpy.sqrt(numpy.multiply(vector_i, vector_j)))))

def MDPA(vector_i, vector_j):
    sum_m = 0
    for m in len(vector_i):
        sum_n = 0
        for n in range(0, m):
            sum_n += vector_i[n] - vector_j[n]
        sum_m += abs(sum_n)

    return sum_m

def cosineSimilarity(vector_i, vector_j):
    return numpy.sum(numpy.multiply(vector_i, vector_j)) / (numpy.sqrt(numpy.sum(vector_i)) * (numpy.sqrt(numpy.sum(vector_j))))

def hammingDistance(vector_i, vector_j):
    sum = 0
    for k in len(vector_i):
        sum += 1 if (vector_i[k] == vector_j[k]) else 0 

    return sum

Modification des données

In [ ]:
result_mc = dict()
result_lbp = dict()

for path_query in query:
    #mc_query = desc_MC()
    lbp_query = desc_LBP(path_query)

    #mc_hist_query, bins = numpy.histogram(mc_query, bins = range(256))
    lbp_hist_query, bins = numpy.histogram(lbp_query, bins = range(256))
    

    for path_db in database:
        # mc_db = desc_MC()
        lbp_db = desc_LBP(path_db)

        #mc_hist_db, bins = numpy.histogram(mc_db, bins = range(256))
        lbp_hist_db, bins = numpy.histogram(lbp_db, bins = range(256))

        # TODO Normaliser les historgrammes

        result_id = path_query[7:-4] + ":" + path_db[16:-4]

        #result_mc[result_id] = numpy.sum(numpy.absolute(mc_hist_query - mc_hist_db))
        result_lbp[result_id] = numpy.sum(numpy.absolute(lbp_hist_query - lbp_hist_db))
    break

print(sorted(result_lbp.items(), key=lambda kv: kv[1]))